In [1]:
from keras.applications import InceptionV3
IMG_WIDTH = 299
IMG_HEIGHT = 299
LABEL = {'athlete':0,'celebrity':1,'ceo':2,'crime':3,'professor':4}
base_model = InceptionV3(weights='imagenet', include_top=False,input_shape=(IMG_WIDTH,IMG_HEIGHT,3))

Using TensorFlow backend.


In [2]:
base_model.summary()

__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            (None, 299, 299, 3)  0                                            
__________________________________________________________________________________________________
conv2d_1 (Conv2D)               (None, 149, 149, 32) 864         input_1[0][0]                    
__________________________________________________________________________________________________
batch_normalization_1 (BatchNor (None, 149, 149, 32) 96          conv2d_1[0][0]                   
__________________________________________________________________________________________________
activation_1 (Activation)       (None, 149, 149, 32) 0           batch_normalization_1[0][0]      
__________________________________________________________________________________________________
conv2d_2 (

In [3]:
from keras.layers import Dense,GlobalAveragePooling2D,Lambda
from keras.models import Model
from keras.optimizers import Adadelta

MODEL_CLASSES = 5
x = base_model.output
x = GlobalAveragePooling2D(name='avg_pool')(x)
# x = Dropout(0.4)(x)
x = Dense(2048,activation='relu')(x)
for i in range(MODEL_CLASSES):
    hidden = Dense(2048,activation='relu')(x)
athlete = Dense(1, activation='sigmoid',name='athlete')(hidden)
celebrity = Dense(1, activation='sigmoid',name='celebrity')(hidden)
ceo = Dense(1, activation='sigmoid',name='ceo')(hidden)
crime = Dense(1, activation='sigmoid',name='crime')(hidden)
professor = Dense(1, activation='sigmoid',name='professor')(hidden)
# hidden = Dense(2048, activation='relu')(x)
# hidden = Dense(2048, activation='relu')(hidden)
# hidden = Dense(2048, activation='relu')(hidden)
# hidden = Dense(2048, activation='relu')(hidden)
# hidden = Dense(2048, activation='relu')(hidden)
# predictions = Dense(units=MODEL_CLASSES, activation='sigmoid')(hidden )
# output_1 = Lambda(lambda x: x[...,:1])(predictions)
# output_2 = Lambda(lambda x: x[...,1:2])(predictions)
# output_3 = Lambda(lambda x: x[...,2:3])(predictions)
# output_4 = Lambda(lambda x: x[...,3:4])(predictions)
# output_5 = Lambda(lambda x: x[...,4:])(predictions)
model = Model(inputs=base_model.input, outputs=[athlete,celebrity,ceo,crime,professor])

adad = Adadelta()

In [4]:
for layer in base_model.layers:
    layer.trainable = False

In [5]:
# model.compile(optimizer='rmsprop',
#               loss='binary_crossentropy',
#               metrics=['accuracy'])
model.compile(optimizer=adad, loss=['binary_crossentropy','binary_crossentropy','binary_crossentropy','binary_crossentropy','binary_crossentropy',],metrics=['accuracy'])

In [6]:
model.summary()

__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            (None, 299, 299, 3)  0                                            
__________________________________________________________________________________________________
conv2d_1 (Conv2D)               (None, 149, 149, 32) 864         input_1[0][0]                    
__________________________________________________________________________________________________
batch_normalization_1 (BatchNor (None, 149, 149, 32) 96          conv2d_1[0][0]                   
__________________________________________________________________________________________________
activation_1 (Activation)       (None, 149, 149, 32) 0           batch_normalization_1[0][0]      
__________________________________________________________________________________________________
conv2d_2 (

In [10]:
import os
import numpy as np
from keras.preprocessing import image
from keras.preprocessing.image import ImageDataGenerator
import scipy
from keras.utils import np_utils

base_dir = './dataset/'
train_dir = os.path.join(base_dir, 'train')
validation_dir = os.path.join(base_dir, 'validation')
test_dir = os.path.join(base_dir, 'test')

def load_img(dir):
    x_list = []
    y_list = []
    for file in sorted(os.listdir(dir)):
        x_list.append(image.load_img(os.path.join(dir,file)))
        tmp = [[0],[0],[0],[0],[0]]
        tmp[LABEL[file.split('_')[0]]] = [1]
        y_list.append(tmp)
    return x_list,y_list

x_train,y_train = load_img(train_dir)
x_val,y_val = load_img(validation_dir)
x_test,y_test = load_img(test_dir)

y_train = np.asarray(y_train,dtype='float32')
y_val = np.asarray(y_val,dtype='float32')
y_test = np.asarray(y_test,dtype='float32')

# y_train = np_utils.to_categorical(y_train, MODEL_CLASSES)
# y_val = np_utils.to_categorical(y_val, MODEL_CLASSES)
# y_test = np_utils.to_categorical(y_test, MODEL_CLASSES)



print(x_test)
print(y_test)
print(type(x_test[0]))

def modify_xshape(data):
    new_shape = (IMG_WIDTH,IMG_HEIGHT,3)
    data_new = np.empty(shape=(len(data),)+new_shape)
    for idx in range(len(data)):
        data_new[idx] = scipy.misc.imresize(data[idx], new_shape)
    return data_new

# def modify_yshape(data):
#     new_shape = (MODEL_CLASSES,1)
#     data_new = np.empty(shape=(len(data),)+new_shape)
#     for idx in range(len(data)):
#         data_new[idx] = data[idx].reshape(MODEL_CLASSES,1)
#     return data_new

x_train = modify_xshape(x_train)
x_val = modify_xshape(x_val)
x_test = modify_xshape(x_test)

# y_train = modify_xshape(y_train)
# y_val = modify_xshape(y_val)
# y_test = modify_xshape(y_test)

# train_dir = os.path.join(base_dir, 'train')
# validation_dir = os.path.join(base_dir, 'validation')
# test_dir = os.path.join(base_dir, 'test')

BATCH_SIZE = 1
EPOCHS = 10

train_datagen = ImageDataGenerator(
    rotation_range=25, 
    width_shift_range=0.1,
    height_shift_range=0.1, 
    shear_range=0.2, 
    zoom_range=0.2,
    horizontal_flip=True,
    rescale=1./255,
    fill_mode="nearest")

# 검증 데이터는 증식되어서는 안 됩니다!
test_datagen = ImageDataGenerator(rescale=1./255)

[<PIL.Image.Image image mode=RGB size=138x138 at 0x7F71A73D6BA8>, <PIL.Image.Image image mode=RGB size=155x155 at 0x7F71A74F0F60>, <PIL.Image.Image image mode=RGB size=174x174 at 0x7F71A74F0F98>, <PIL.Image.Image image mode=RGB size=143x143 at 0x7F71A74F0E10>, <PIL.Image.Image image mode=RGB size=162x162 at 0x7F71A74F0DA0>, <PIL.Image.Image image mode=RGB size=188x188 at 0x7F71A74F0D30>, <PIL.Image.Image image mode=RGB size=147x147 at 0x7F71A74F0CF8>, <PIL.Image.Image image mode=RGB size=149x149 at 0x7F71A74F0C88>, <PIL.Image.Image image mode=RGB size=133x133 at 0x7F71A74F0278>, <PIL.Image.Image image mode=RGB size=174x174 at 0x7F71A74F0C18>, <PIL.Image.Image image mode=RGB size=107x107 at 0x7F71A74E7EB8>, <PIL.Image.Image image mode=RGB size=110x110 at 0x7F71A74E7FD0>, <PIL.Image.Image image mode=RGB size=313x313 at 0x7F71A74E7EF0>, <PIL.Image.Image image mode=RGB size=339x339 at 0x7F71A74E71D0>, <PIL.Image.Image image mode=RGB size=121x121 at 0x7F71A74E7E48>, <PIL.Image.Image image m

/home/minkj1992/anaconda3/envs/keras/lib/python3.6/site-packages/ipykernel_launcher.py:45: DeprecationWarning: `imresize` is deprecated!
`imresize` is deprecated in SciPy 1.0.0, and will be removed in 1.3.0.
Use Pillow instead: ``numpy.array(Image.fromarray(arr).resize())``.


In [11]:
# def extract_features(datagen,x,y):
#     return datagen.flow(x,y,batch_size=BATCH_SIZE)

def generate_data_generator(generator,x,y1,y2,y3,y4,y5):
    genX = generator.flow(x, seed=7)
    genY1 = generator.flow(y1, seed=7)
    genY2 = generator.flow(y2, seed=7)
    genY3 = generator.flow(y3, seed=7)
    genY4 = generator.flow(y4, seed=7)
    genY5 = generator.flow(y5, seed=7)
    while True:
            Xi = genX.next()
            Yi1 = genY1.next()
            Yi2 = function(Y2)
            yield Xi, [Yi1, Yi2]

train_generator = extract_features(train_datagen,x_train,y_train)
valid_generator = extract_features(test_datagen,x_val,y_val)
test_generator = extract_features(test_datagen,x_test,y_test)



In [12]:
history = model.fit_generator(generator=train_generator,
                    steps_per_epoch=len(x_train) / BATCH_SIZE,
                    validation_data=valid_generator,
                    validation_steps=len(x_val) / BATCH_SIZE,
                    epochs=EPOCHS)
MODEL_FILE = 'inception_v3_face.h5'

model.save(MODEL_FILE)

Epoch 1/10


ValueError: Error when checking model target: the list of Numpy arrays that you are passing to your model is not the size the model expected. Expected to see 5 array(s), but instead got the following list of 1 arrays: [array([[[0.],
        [0.],
        [0.],
        [0.],
        [1.]]], dtype=float32)]...

In [ ]:
import matplotlib.pyplot as plt
def smooth_curve(points, factor=0.8):
    smoothed_points = []
    for point in points:
        if smoothed_points:
            previous = smoothed_points[-1]
            smoothed_points.append(previous * factor + point * (1 - factor))
        else:
            smoothed_points.append(point)
    return smoothed_points

In [ ]:
acc = history.history['acc']
val_acc = history.history['val_acc']
loss = history.history['loss']
val_loss = history.history['val_loss']

epochs = range(len(acc))

plt.plot(epochs,
         smooth_curve(acc), 'bo', label='Smoothed training acc')
plt.plot(epochs,
         smooth_curve(val_acc), 'b', label='Smoothed validation acc')
plt.title('Training and validation accuracy')
plt.legend()

plt.figure()

plt.plot(epochs,
         smooth_curve(loss), 'bo', label='Smoothed training loss')
plt.plot(epochs,
         smooth_curve(val_loss), 'b', label='Smoothed validation loss')
plt.title('Training and validation loss')
plt.legend()

plt.show()

In [18]:
y_train.shape

(500, 5, 1)